In [1]:
import pandas as pd
import pandapipes as pp
import numpy as np

## Simulation variables and options

# Ambient temperature [K] = t[C] + 273.15
temp_ext_c = 20
temp_ext_k = temp_ext_c + 273.15

# Initial network fluid temperature (flow side) [K] = t[C] + 273.15
t_net_flow_init_c = 85
t_net_flow_init_k = t_net_flow_init_c + 273.15

# Initial network fluid temperature (return side) [K] = t[C] + 273.15
t_net_return_init_c = 35
t_net_return_init_k = t_net_return_init_c + 273.15

# Initial junction and network pressure, flow side [bar]
net_flow_p_bar = 10

# Initial junction and network pressure, return side [bar]
net_return_p_bar = 8

In [2]:
## Import data from Data.xls into Pandas dataframes

# Define data file path

# Import data
def import_data(sourcefile):
    df_heater = pd.read_excel(sourcefile, sheet_name=0)
    df_sink = pd.read_excel(sourcefile, sheet_name=1)
    df_connection = pd.read_excel(sourcefile, sheet_name=2)
    df_nodetype = pd.read_excel(sourcefile, sheet_name=3)
    return df_heater,df_sink,df_connection,df_nodetype
# Prepare blank network for elements
net = pp.create_empty_network(name="Data", fluid="water")

In [3]:
## Create pandapipes junctions from imported data.
# Creates separate networks for supply and return lines. Supply and return
# lines join at sinks and heat sources.
def create_junctions(net,global_arr,df_nodetype):
    # Get number of nodes from dataframe
    num_nodes = len(df_nodetype)

    # tracker tracks number of junctions created
    junction_tracker = 0

    # Iterate over all nodes to find and create junctions
    for i in range(num_nodes):

        if df_nodetype.at[i, 'Node Type'] == "Junction":
            
            # Replace dashes with underscores to conform with Python markup
            junction_name = str(df_nodetype.at[i, 'Name']).replace("Junction-",
                                                                "Junction_")
            
            junction_pos_x = float(df_nodetype.at[i, 'X-Coordinate'])
            junction_pos_y = float(df_nodetype.at[i, 'Y-Coordinate'])
            junction_pos_flow = (junction_pos_x, junction_pos_y)
            junction_pos_return = (junction_pos_x, junction_pos_y-100)

            # Define supply and return flow network junctions
            global_arr['%s_supply' % junction_name] = pp.create_junction(net,
                pn_bar=net_flow_p_bar,
                tfluid_k=t_net_flow_init_k,
                geodata=junction_pos_flow,
                name=junction_name + '_supply')
            global_arr['%s_return' % junction_name] = pp.create_junction(net,
                pn_bar=net_return_p_bar,
                tfluid_k=t_net_return_init_k,
                geodata=junction_pos_return,
                name=junction_name + '_return')

            junction_tracker += 2

    print("Number of junctions created:", junction_tracker)

    # List junctions
    #net.junction

In [ ]:
## Create pandapipes source nodes

# See markdown notes above for information on multiple producer networks.

# Raw source heater values are given as kW [J/s * 10^3], pp circ_pump_mass
# uses mass flow mkg/s, use conversion ratio variable to adjust.
def create_source_node(net,global_arr,df_heater,df_connection):
    # Conversion ratio from raw data, denotes average ratio between base demand
    # [kW] and base demand [kg/s].  Assumes ~50 K / kg of energy heat difference
    # between flow and return networks (210.4648 * 10^3 J/kg / 4190 J/kg ~= 50).
    e_flow_conv_ratio = (7 / 0.033 + 40.3 / 0.193) / 2 # = 210.4648
    #e_flow_conv_ratio = 41.9 #_______________
    # Output water temperature [K] = t[C] + 273.15
    t_out_c = 90
    t_out_k = t_out_c + 273.15

    # Pressure at flow side of producers [bar]
    source_flow_bar = 10

    # Pressure lift induced by sources [bar] - Used to model pressure difference
    # at pump nodes between input and output
    pressure_lift = 3

    # Production multiplier - general multiplier for testing effects of increasing/
    # reducing heated water production. Multiplier applies to mdot_flow_kg_per_s
    prod_multiplier = 1

    # Get source connections and create source nodes
    for i in range(len(df_heater)):
        
        if str(df_heater.iloc[i,0]) != "Ericsson": # Ericsson special case below
            source_name = str(df_heater.iloc[i,0])

            for j in range(len(df_connection)):
                
                if str(df_connection.at[j, 'Start Node']) == source_name:
                    
                    source_flow = global_arr[str(df_connection.at[j, 'End Node'])
                                            .replace("Junction-", "Junction_") + 
                                            '_supply']
                    
                    source_return = global_arr[str(df_connection.at[j, 'End Node'])
                                            .replace("Junction-", "Junction_") +
                                            '_return']
                    
                    pp.create_circ_pump_const_pressure(
                        net,
                        flow_junction=source_flow,
                        return_junction=source_return,
                        plift_bar=pressure_lift,
                        p_flow_bar=source_flow_bar,
                        mdot_flow_kg_per_s=pd.to_numeric(df_heater.iloc[i,2])/e_flow_conv_ratio,
                        t_flow_k=t_out_k,
                        name=source_name)
            
                elif str(df_connection.at[j, 'End Node']) == source_name:

                    source_flow = global_arr[str(df_connection.at[j, 'Start Node'])
                                            .replace("Junction-", "Junction_") + 
                                            '_supply']
                    
                    source_return = global_arr[str(df_connection.at[j, 'Start Node'])
                                            .replace("Junction-", "Junction_") +
                                            '_return']
                    
                    pp.create_circ_pump_const_pressure(
                        net,
                        flow_junction=source_flow,
                        return_junction=source_return,
                        plift_bar=pressure_lift,
                        p_flow_bar=source_flow_bar,
                        #mdot_flow_kg_per_s=pd.to_numeric(df_heater.iloc[i,2])/e_flow_conv_ratio,
                        t_flow_k=t_out_k,
                        name=source_name)

    # Ericsson special case: Ericsson has only return connections;
    # Ericsson set as a slack node in the network.

    # Create auxiliary junction to connect to Ericsson

    global_arr['Ericsson_connection'] = pp.create_junction(net, 
                                        pn_bar=net_return_p_bar,
                                        tfluid_k=t_net_return_init_k,
                                        name='Ericsson_connection')
    Ericsson = pp.create_ext_grid(net,
                                junction=global_arr['Ericsson_connection'],
                                p_bar=net_return_p_bar,
                                t_k=t_net_return_init_k,
                                name="Ericsson Ext. Grid")

    # Tracker for Ericsson auxiliary junction connections
    e_pipe_tracker = 0

    for i in range(len(df_connection)):

        if str(df_connection.at[i, 'Start Node']) == 'Ericsson':
                    
            connection_name = global_arr[str(df_connection.at[i, 'End Node'])
                                            .replace("Junction-", "Junction_") + 
                                            '_return']

            pp.create_pipe_from_parameters(net,
                from_junction=connection_name,
                to_junction=global_arr['Ericsson_connection'],
                length_km=float(str(df_connection.at[i, 'Length [m]']))/1000,
                diameter_m=float(str(df_connection.at[i, 'Diameter [mm]']))/1000,
                k_mm=.05,
                alpha_w_per_m2k=float(str(df_connection.at[i, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                text_k=int(temp_ext_k),
                name='Pipe_E_' + str(e_pipe_tracker)
                )
        
            e_pipe_tracker += 1
            
        elif str(df_connection.at[i, 'End Node']) == 'Ericsson':

            connection_name = global_arr[str(df_connection.at[i, 'Start Node'])
                                            .replace("Junction-", "Junction_") + 
                                            '_return']

            pp.create_pipe_from_parameters(net,
                from_junction=connection_name,
                to_junction=global_arr['Ericsson_connection'],
                length_km=float(str(df_connection.at[i, 'Length [m]']))/1000,
                diameter_m=float(str(df_connection.at[i, 'Diameter [mm]']))/1000,
                k_mm=.05,
                alpha_w_per_m2k=float(str(df_connection.at[i, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                text_k=int(temp_ext_k),
                name='Pipe_E_' + str(i)
                )

            e_pipe_tracker += 1

    # Create Ericsson slack node
    #print(net.ext_grid) 
    # List sources
    #net.circ_pump_pressure
    #help(pp.create_pipe_from_parameters)

In [5]:
## Create pandapipes sink nodes
def create_sink_nodes(net,global_arr,df_sink,df_connection,timestep):
    # Desired return water temperature [K] = t[C] + 273.15
    # Use t_return_c = t_net_return_init_c to set to initial network temp
    t_return_c = t_net_return_init_c + 5
    t_return_k = t_return_c + 273.15

    # Get number of sinks and connections
    num_sinks = len(df_sink)
    num_connections = len(df_connection)

    # Tracker tracks number of sinks created
    sink_tracker = 0

    # Iterate over number of raw data rows to find and create sinks
    for i in range(num_sinks):

        sink_get = str(df_sink.at[i, 'Name'])

        # Get sink details
        for j in range(num_sinks):
            if df_sink.at[j, 'Name'] == sink_get:
                sink_type = str(df_sink.at[j, 'Category'])
                sink_priority = int(str(df_sink.at[j, 'Priority']))
                sink_demand_kW = float(str(df_sink.at[j, 'Base Demand [kW]']))
                mdot_kg_per_s_demand_timestep = float(str(df_sink.iloc[j,6+timestep]))
                mdot_kg_per_s_demand = float(str(df_sink.at[j, 'Base Demand [kg/s]']))
                sink_x = float(str(df_sink.at[j, 'X-Coordinate']))
                sink_y = float(str(df_sink.at[j, 'Y-Coordinate']))

        # Get connecting junction and pipe diameter
        for k in range(num_connections):
            if df_connection.at[k, "End Node"] == sink_get:
                sink_source = global_arr[str(df_connection.at[k, 'Start Node']).
                                        replace("Junction-", "Junction_") + '_supply']
                sink_return = global_arr[str(df_connection.at[k, 'Start Node']).
                                        replace("Junction-", "Junction_") + '_return']
                connection_d_mm = float(str(df_connection.at[k, 'Diameter [mm]']))
                mdot_kg_per_s_capacity = float(str(df_connection.at[k, 'Capacity [kg/s]']))

        sink_name = sink_get.replace("Junction-", "Sink_")

        # Define sink
        pp.create_heat_consumer(net,
                                from_junction=sink_source,
                                to_junction=sink_return,
                                diameter_m=connection_d_mm/1000,
                                qext_w=sink_demand_kW*1000,
                                #controlled_mdot_kg_per_s=mdot_kg_per_s_capacity,
                                controlled_mdot_kg_per_s=(mdot_kg_per_s_demand+mdot_kg_per_s_demand_timestep)/10.0,
                                #deltat_k=50,
                                #treturn_k=t_return_k,
                                name=sink_name, type=sink_type,
                                x = sink_x, y = sink_y)
            
        sink_tracker += 1

    #help(pp.create_heat_consumer)
    print("Number of sinks created:", sink_tracker)
    #net.heat_consumer

In [6]:
## Create pipe connections

# Defines pipe connections between junctions. Source and sink connections
# are defined at heat pump node and heat exchanger creation.
def create_pipe_connections(net,global_arr,df_heater,df_sink,df_connection):
    # Tracker tracks number of pipes created
    pipe_tracker = 0

    # Concatenate source and sink names to single NumPy array
    np_heaters_sinks = pd.concat([df_heater[['Name']], df_sink[['Name']]]).to_numpy()
    num_heaters_sinks = len(np_heaters_sinks)
    num_connections = len(df_connection)
    for i in range(num_connections):
        
        pipe_get = str(df_connection.at[i, 'Name']).replace("-", "_")

        start_node = str(df_connection.at[i, 'Start Node'])
        end_node = str(df_connection.at[i, 'End Node'])

        if str(df_connection.at[i, 'Has Supply Line']) == 'True': 
            #print(1)
            supply_line = True
        else: 
            supply_line = False
        if str(df_connection.at[i, 'Has Return Line']) == 'True':
            #print(-1) 
            return_line = True
        else: 
            return_line = False
    
        # Exclude source and sink node connections (defined earlier)
        if start_node not in np_heaters_sinks and end_node not in np_heaters_sinks:
                
            # Create supply line pipe
            if supply_line == True:
                
                pipe_from = global_arr[str(df_connection.at[i, 'Start Node']).
                                    replace("Junction-", "Junction_") + '_supply']
                pipe_to = global_arr[str(df_connection.at[i, 'End Node']).
                                    replace("Junction-", "Junction_") + '_supply']
                pipe_name = pipe_get + '_supply'

                pp.create_pipe_from_parameters(net,
                    from_junction=pipe_from,
                    to_junction=pipe_to,
                    length_km=float(str(df_connection.at[i, 'Length [m]']))/1000,
                    diameter_m=float(str(df_connection.at[i, 'Diameter [mm]']))/1000,
                    k_mm=.05,
                    u_w_per_m2k=float(str(df_connection.at[i, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                    sections=5,
                    text_k=int(temp_ext_k),
                    name=pipe_name
                    )
                pipe_tracker += 1
            
            # Create return line pipe
            if return_line == True:

                pipe_from = global_arr[str(df_connection.at[i, 'End Node']).
                    replace("Junction-", "Junction_") + '_return']
                pipe_to = global_arr[str(df_connection.at[i, 'Start Node']).
                    replace("Junction-", "Junction_") + '_return']
                pipe_name = pipe_get + '_return'

                pp.create_pipe_from_parameters(net,
                    from_junction=pipe_from,
                    to_junction=pipe_to,
                    length_km=float(str(df_connection.at[i, 'Length [m]']))/1000,
                    diameter_m=float(str(df_connection.at[i, 'Diameter [mm]']))/1000,
                    u_w_per_m2k=float(str(df_connection.at[i, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                    text_k=int(temp_ext_k),
                    name=pipe_name,
                    sections=5,
                    k_mm=.05
                    )
                pipe_tracker += 1

    print("Number of pipes created: ", pipe_tracker)
    #net.pipe

In [7]:
## Run pipeflow
global_arr={}
sourcefile = './data/Data.xlsx'
df_heater,df_sink,df_connection,df_nodetype=import_data(sourcefile)
create_junctions(net,global_arr,df_nodetype)
create_source_node(net,global_arr,df_heater,df_connection)

timestep = 3
create_sink_nodes(net,global_arr,df_sink,df_connection,timestep) 
create_pipe_connections(net,global_arr,df_heater,df_sink,df_connection)
pp.pipeflow(net, mode='bidirectional') # Modes = sequential, bidirectional

#print(net.res_pipe) # Modes = res_heat_consumer, res_junction, res_pipe, res_circ_pump_pressure

Number of junctions created: 1588
Number of sinks created: 496
Number of pipes created:  1616


In [ ]:
damage_pipes=[]
for i in range(len(net.res_pipe)):
    name=(net.pipe).at[i,'name']
    if name.endswith('_return'):
        name=name.removesuffix('_return')
    j=0
    while j<len(df_connection):
        tname=df_connection.iloc[j,0].replace('Pipe-','Pipe_')
        if tname==name:
            #print(net.res_pipe.at[i,'mdot_from_kg_per_s']*4.190*50,end="  ")
            #print(df_connection.iloc[j,8])
            if abs(net.res_pipe.at[i,'mdot_from_kg_per_s'])*4.190*50>df_connection.iloc[j,8]:
                damage_pipes.append(name)
        j+=1

334.18389998169323  3354
777.0559496519892  11253
777.0559496519892  11253
6.479650000000749  321
1111.2398496336825  11253
272.9251989685556  11253
3942.238871969026  5533
3.434100000000431  172
1.1446999999992793  172
2.289400000001771  60
1.3285000000015554  60
2.4731999999991543  60
2.4731999999991543  60
2.289400000001771  60
5.907300000001943  172
7.235800000002881  172
9.709000000001415  172
10.465150000000573  172
-4.5788000000036  172
-8.407771893338706  5533
-1.2966988593987596e-12  172
-7720.998943956079  20264
-1.6354862337641212  60
-10.338317620278172  321
113.80829069398884  5533
320.75085049743103  3354
15.875300018310256  321
-840.6897316394248  5533
49.71055005491983  1915
49.71055005491943  962
7.8081500000002535  630
20.5627000000003  630
-4.845406697954949  630
-789.5198005156411  11253
130.62686582736544  11253
185.08473842928987  11253
130.62686582736637  5533
4041.3468212244024  11253
4091.0573712793207  11253
4041.3468212244024  11253
71.47239996339222  962
71.

In [27]:
print(damage_pipes)

['Pipe_7666', 'Pipe_34012', 'Pipe_51587']
